<a href="https://colab.research.google.com/github/PolinaLinaNechaeva/Data-Business-Analytics-Dashboard/blob/main/Analyst_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

# Load dataset from LinkedIn scraper (JSON file)
# Replace with your exact path if different
file_path = "dataset_linkedin-jobs-scraper_2025-11-19_12-03-47-361.json"

# Read JSON
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert JSON list into a DataFrame
df = pd.DataFrame(data)

# --- Add helper columns ---
# Extract job title
df["job_title"] = df["title"]

# Extract company name (depends on scraper structure)
df["company_name"] = df.get("companyName")

# Extract job URL
df["job_url"] = df.get("jobUrl")

# Extract location
df["location"] = df.get("location")

# Extract date posted
df["date_posted"] = pd.to_datetime(df.get("listedAt"), errors="coerce")

# Extract description (fallback if descriptionText or descriptionHtml exist)
if "description" in df.columns:
    df["description_clean"] = df["description"]
elif "descriptionText" in df.columns:
    df["description_clean"] = df["descriptionText"]
elif "descriptionHtml" in df.columns:
    df["description_clean"] = df["descriptionHtml"]
else:
    df["description_clean"] = None

# Add job source (useful if combining sources later)
df["source"] = "LinkedIn"

print("Rows loaded:", len(df))
print(df.head())
print(df.info())

# Save cleaned version
df.to_csv("linkedin_jobs_clean_2025-11-19.csv", index=False, encoding="utf-8-sig")

Rows loaded: 932
                                          title  \
0                 Business Intelligence Analyst   
1  Senior/Staff Data Analyst - People Analytics   
2                          Data Analyst (m/w/d)   
3                         Business Data Analyst   
4                                  Data Analyst   

                                 location   postedTime publishedAt  \
0                                 Germany  2 weeks ago  2025-10-30   
1                 Berlin, Berlin, Germany   6 days ago  2025-11-13   
2  Frankfurt Rhine-Main Metropolitan Area   1 week ago  2025-11-12   
3         North Rhine-Westphalia, Germany   1 week ago  2025-11-12   
4   Stuttgart, Baden-Württemberg, Germany  3 weeks ago  2025-10-27   

                                              jobUrl    companyName  \
0  https://de.linkedin.com/jobs/view/business-int...          Codex   
1  https://de.linkedin.com/jobs/view/senior-staff...  Delivery Hero   
2  https://de.linkedin.com/jobs/view/data-

In [ ]:
import re
import pandas as pd

# 0) Load unified cleaned LinkedIn file
df = pd.read_csv("linkedin_jobs_clean_2025-11-19.csv")

# ----------------------------------------------------------
# 1) Base variables
# ----------------------------------------------------------
# English-language job titles, simple and clean.
# We ignore gender and German inflections here.

# Exact roles we want:
#   - Data Analyst
#   - Business Analyst
#   - Data Analytics roles
#   - BI Analyst (optional)
#   - Data Insights Analyst / Reporting Analyst (optional)

ANALYST = r"(analyst|analytics)"

DATA = r"(data)"
BUSINESS = r"(business)"
BI = r"(bi|business intelligence)"
INSIGHTS = r"(insights?|reporting)"

# ----------------------------------------------------------
# 2) Patterns for role classification
# ----------------------------------------------------------
# Each pattern is a named group → same logic as your old code.
grp_patterns = {
    "data_analyst":        rf"{DATA}\s*{ANALYST}",
    "business_analyst":    rf"{BUSINESS}\s*{ANALYST}",
    "bi_analyst":          rf"{BI}\s*{ANALYST}",
    "data_reporting":      rf"{DATA}.*({INSIGHTS}|reporting)",
    "analytics_role":      rf"{ANALYST}",   # fallback for generic analyst roles
}

# ----------------------------------------------------------
# 3) Build combined regex with named groups
# ----------------------------------------------------------
named_parts = [f"(?P<{k}>{v})" for k, v in grp_patterns.items()]
pattern = re.compile("|".join(named_parts), flags=re.IGNORECASE)

# ----------------------------------------------------------
# 4) Function to detect matched group
# ----------------------------------------------------------
def match_group(text: str):
    if not isinstance(text, str):
        return None
    m = pattern.search(text)
    return m.lastgroup if m else None

# ----------------------------------------------------------
# 5) Apply classification on job titles
# ----------------------------------------------------------
search_series = df["job_title"].astype(str)
hit_group = search_series.apply(match_group)

df_filtered = df.loc[hit_group.notna()].copy()
df_filtered["job_match_group"] = hit_group.loc[df_filtered.index].values

print("Total rows:", len(df))
print("Matched:", len(df_filtered))
print(df_filtered["job_match_group"].value_counts())

# ----------------------------------------------------------
# 6) Remove duplicates by job URL
# ----------------------------------------------------------
for col in ["job_url", "url", "jobUrl", "JobURL", "Job URL"]:
    if col in df_filtered.columns:
        before = len(df_filtered)
        df_filtered = df_filtered.drop_duplicates(subset=[col])
        print(f"Duplicates removed by {col}: {before - len(df_filtered)}")
        break

# ----------------------------------------------------------
# 7) Preview final result
# ----------------------------------------------------------
cols_preview = [
    c for c in ["job_match_group","job_title","title","description_clean","location"]
    if c in df_filtered.columns
]
print(df_filtered[cols_preview].head(10))

# ----------------------------------------------------------
# 8) Save result
# ----------------------------------------------------------
df_filtered.to_excel("linkedin_jobs_filtered_2025-11-19.xlsx", index=False)

print(f"Final number of rows: {len(df_filtered)}")

Total rows: 932
Matched: 485
job_match_group
data_analyst        245
analytics_role      181
business_analyst     34
bi_analyst           16
data_reporting        9
Name: count, dtype: int64
Duplicates removed by job_url: 0
  job_match_group                                     job_title  \
0      bi_analyst                 Business Intelligence Analyst   
1    data_analyst  Senior/Staff Data Analyst - People Analytics   
2    data_analyst                          Data Analyst (m/w/d)   
3    data_analyst                         Business Data Analyst   
4    data_analyst                                  Data Analyst   
5    data_analyst                     Data Analyst (all gender)   
6    data_analyst                Sales Data Analyst (Part-Time)   
7  analytics_role         Internship - Data & Analytics (m/f/x)   
8    data_analyst                           Data Analyst - D365   
9    data_analyst                          Data Analyst (f/m/d)   

                                      

In [ ]:
import re
import pandas as pd
import numpy as np

# Make a working copy (cleaning happens here)
dfc = df_filtered.copy()

# -------------------------------------------------------------
# 1) Normalize spaces ONLY in string/object columns
#    - remove invisible characters
#    - collapse multiple spaces
#    - strip leading/trailing spaces
# -------------------------------------------------------------
for col in dfc.columns:
    if dfc[col].dtype == object:
        dfc[col] = dfc[col].apply(
            lambda x: re.sub(r"\s+", " ", x).strip() if isinstance(x, str) else x
        )

# -------------------------------------------------------------
# 2) Remove clearly empty / useless columns
#    Some LinkedIn scrapers include optional fields (benefits, applyUrl, companyLogo)
#    If a column contains only null/empty values → we drop it
# -------------------------------------------------------------
drop_cols = []

# Drop "benefits" column if it is completely empty
if "benefits" in dfc.columns and dfc["benefits"].nunique(dropna=True) == 0:
    drop_cols.append("benefits")

# Drop empty "companyLogo" if scraper included it
if "companyLogo" in dfc.columns and dfc["companyLogo"].nunique(dropna=True) == 0:
    drop_cols.append("companyLogo")

# Drop empty "applyUrl" if scraper included it
if "applyUrl" in dfc.columns and dfc["applyUrl"].nunique(dropna=True) == 0:
    drop_cols.append("applyUrl")

# Apply the drop
dfc = dfc.drop(columns=drop_cols, errors="ignore")

print("Dropped columns:", drop_cols)
print("Resulting shape:", dfc.shape)

Dropped columns: ['benefits']
Resulting shape: (485, 26)


In [ ]:
import re

# -------------------------------------------------------------
# Regular expressions for detecting seniority level
# Adapted for LinkedIn Data/BI Analyst roles
# -------------------------------------------------------------

# Senior-level patterns
RE_SENIOR = re.compile(
    r'\b(senior|sr\.?|sen\.?|lead|team\s*lead|principal|head|staff|expert'
    r'|leitung|leiter(?:in)?)\b',
    re.I
)

# Junior-level patterns
RE_JUNIOR = re.compile(
    r'\b(junior|jr\.?|entry|graduate|einsteiger|absolvent|trainee|'
    r'werkstudent|working\s*student|praktikant(?:in)?)\b',
    re.I
)

# Mid-level patterns
RE_MID = re.compile(
    r'\b(mid(?:dle)?|medior|intermediate|professional|experienced|regular)\b',
    re.I
)

# -------------------------------------------------------------
# Function: canonical seniority detection
# -------------------------------------------------------------
def canon_experience_from_text(text: str) -> str | None:
    """
    Detects seniority level (Junior / Mid / Senior) from text.
    Works on LinkedIn titles & descriptions.
    Returns:
        "Junior", "Mid", "Senior" or None if no clues found.
    """
    if text is None or not isinstance(text, str):
        return None

    # Normalize text for pattern matching
    s = re.sub(r'[/,_-]+', ' ', text.lower()).strip()

    # Exclude false positives (e.g., 'Senioren' in German)
    s = re.sub(r'\bsenioren\b', ' ', s)

    # Boolean flags for detected patterns
    has_sen = bool(RE_SENIOR.search(s))
    has_jun = bool(RE_JUNIOR.search(s))
    has_mid = bool(RE_MID.search(s))

    # Resolve conflicts or ambiguous cases
    if has_sen and not has_jun:
        return "Senior"
    if has_jun and not has_sen:
        return "Junior"
    if has_sen and has_jun:
        return "Mid"
    if has_mid:
        return "Mid"

    return None

In [ ]:
import re
import pandas as pd

# -------------------------------------------------------------
# Cell 4 — Normalize publishedAt + restore from postedTime-like text, then drop postedTime
# -------------------------------------------------------------

# 0) Fix "today" as a constant date (per project conditions)
#    For this LinkedIn run, we assume today = 2025-11-19
base_date = pd.Timestamp("2025-11-19")  # "today" for this dataset
yesterday = base_date - pd.Timedelta(days=1)


def parse_published_at(s: str):
    """
    Parse publishedAt with different possible formats:
    - dd.mm.yyyy
    - yyyy-mm-dd or ISO-like with time
    - mm/dd/yyyy
    Returns pandas.Timestamp or NaT.
    """
    if pd.isna(s):
        return pd.NaT
    s = str(s).strip()
    if not s:
        return pd.NaT

    # dd.mm.yyyy
    if re.fullmatch(r"\d{1,2}\.\d{1,2}\.\d{2,4}", s):
        return pd.to_datetime(s, format="%d.%m.%Y", errors="coerce")

    # yyyy-mm-dd (ISO, possibly with time)
    if re.fullmatch(r"\d{4}-\d{2}-\d{2}(T.*)?", s):
        return pd.to_datetime(s, errors="coerce")

    # mm/dd/yyyy
    if re.fullmatch(r"\d{1,2}/\d{1,2}/\d{2,4}", s):
        return pd.to_datetime(s, errors="coerce", monthfirst=True)

    # Fallback: let pandas try, prefer day-first interpretation
    return pd.to_datetime(s, errors="coerce", dayfirst=True)


def restore_date_from_posted(txt: str):
    """
    Restore publication date from a relative "postedTime"-like text,
    for example:
      - "today", "yesterday", "just now"
      - "3 days ago", "2 weeks ago", "1 month ago"
      - "5 hours ago" (we treat as 'today')
    All dates are calculated relative to base_date.
    """
    if pd.isna(txt):
        return pd.NaT
    s = str(txt).strip().lower()
    if s in {"", "nan", "none", "null", "na", "n/a"}:
        return pd.NaT

    # today / yesterday / just now
    if "today" in s or "just now" in s:
        return base_date
    if "yesterday" in s:
        return yesterday

    # months
    m = re.search(r"(\d+)\s*month", s)
    if m:
        k = int(m.group(1))
        return base_date - pd.Timedelta(days=30 * k)

    # weeks
    m = re.search(r"(\d+)\s*week", s)
    if m:
        k = int(m.group(1))
        return base_date - pd.Timedelta(days=7 * k)

    # days
    m = re.search(r"(\d+)\s*day", s)
    if m:
        k = int(m.group(1))
        return base_date - pd.Timedelta(days=k)

    # hours → treat as "today"
    m = re.search(r"(\d+)\s*hour", s)
    if m:
        return base_date

    return pd.NaT


# 1) Normalize publishedAt → datetime (if column exists)
if "publishedAt" in dfc.columns:
    dfc["publishedAt"] = dfc["publishedAt"].apply(parse_published_at)

# 2) Restore missing publishedAt from postedTime-like column (if present)
#    For LinkedIn, this might be something like "postedTime", "timeAgo", etc.
if "postedTime" in dfc.columns:
    mask_missing = dfc["publishedAt"].isna()
    if mask_missing.any():
        dfc.loc[mask_missing, "publishedAt"] = (
            dfc.loc[mask_missing, "postedTime"].apply(restore_date_from_posted)
        )

# 3) Drop postedTime helper column if it exists
if "postedTime" in dfc.columns:
    dfc = dfc.drop(columns=["postedTime"])

# 4) Add string representation of publishedAt for export (keep datetime as well)
dfc["publishedAt_str"] = dfc["publishedAt"].dt.strftime("%d.%m.%Y").fillna("")

In [ ]:
import re
import numpy as np
import pandas as pd

# -------------------------------------------------------------
# Helpers for salary parsing
# -------------------------------------------------------------

def _extract_multiplier(s: str) -> int:
    """
    Detects whether the salary is per month or per year based on the text.
    Returns an integer multiplier to convert the value to yearly salary.
      - monthly → * 12
      - yearly → * 1
      - if nothing is clear → assume yearly
    """
    s_low = s.lower()

    # Monthly payment → multiply by 12
    if any(tag in s_low for tag in ["/mo", "/month", "per month", "monat"]):
        return 12

    # Yearly payment → multiplier 1
    if any(tag in s_low for tag in ["/yr", "/year", "per year", "p.a.", "pa", "jahr"]):
        return 1

    # No explicit period → assume yearly salary
    return 1


# Matches thousand separators (1.234 / 1,234 / 1 234)
_thousands_sep_re = re.compile(r"(?<=\d)[\.,\s](?=\d{3}(\D|$))")


def _to_number(token: str) -> float | None:
    """
    Convert a token like '60k', '75.000', '100 000', '90,5' into a float.
    Handles:
      - 'k' and 'm' suffixes
      - different thousand separators
      - commas vs dots as decimal separator
    Returns float or None if cannot parse.
    """
    if not token:
        return None

    t = token.strip().lower()

    # Suffixes: k, m (thousands, millions)
    mult_suffix = 1.0
    if t.endswith("k"):
        mult_suffix = 1_000.0
        t = t[:-1]
    elif t.endswith("m"):
        mult_suffix = 1_000_000.0
        t = t[:-1]

    # Remove currencies and letters, keep only digits, dots and commas
    t = re.sub(r"[^\d\.,]", "", t)

    # Remove thousand separators: dot, comma or space before 3 digits
    t = _thousands_sep_re.sub("", t)

    # Normalize decimal separator: comma → dot
    t = t.replace(",", ".")

    try:
        return float(t) * mult_suffix if t else None
    except ValueError:
        return None


def _parse_one_side(s: str, multiplier_hint: int | None = None) -> float | None:
    """
    Parse one side of the salary range and apply period multiplier.
    Example: '60k / year' → 60000
    """
    if not s:
        return None

    m = _extract_multiplier(s) if multiplier_hint is None else multiplier_hint
    val = _to_number(s)
    return (val * m) if val is not None else None


def split_salary_cell(cell):
    """
    Split salary cell into min and max annual salary in EUR-like units.
    Handles formats like:
      - '60k'
      - '60k - 80k'
      - '60.000 - 80.000 € p.a.'
      - '5 000 / month'
    Returns a Series with:
      - 'salary_min (€)'
      - 'salary_max (€)'
    """
    # Empty values
    if pd.isna(cell):
        return pd.Series({"salary_min (€)": np.nan, "salary_max (€)": np.nan})

    s = str(cell).strip()
    if s.lower() in {"", "nan", "none", "null", "na", "n/a"}:
        return pd.Series({"salary_min (€)": np.nan, "salary_max (€)": np.nan})

    # Common multiplier (if period is specified only once in the text)
    mult = _extract_multiplier(s)

    # Split range by hyphen or en dash
    parts = re.split(r"\s*[-–]\s*", s)
    parts = [p for p in parts if p]

    # Single value → use it as both min and max
    if len(parts) == 1:
        v = _parse_one_side(parts[0], multiplier_hint=mult)
        return pd.Series({"salary_min (€)": v, "salary_max (€)": v})

    # Two values → min and max
    v1 = _parse_one_side(parts[0], multiplier_hint=mult)
    v2 = _parse_one_side(parts[1], multiplier_hint=mult)
    return pd.Series({"salary_min (€)": v1, "salary_max (€)": v2})


# -------------------------------------------------------------
# Apply salary parsing if salary column exists
# -------------------------------------------------------------
if "salary" in dfc.columns:
    salary_split = dfc["salary"].apply(split_salary_cell)

    # Drop original raw salary text
    dfc = dfc.drop(columns=["salary"])

    # Attach parsed numeric columns
    dfc[["salary_min (€)", "salary_max (€)"]] = salary_split

In [ ]:
# -------------------------------------------------------------
# Split 'location' into city / region / country and drop original column
# Typical LinkedIn format: "City, Region, Country"
# Examples:
#   "Berlin, Berlin, Germany"
#   "Munich, Bavaria, Germany"
#   "Hamburg, Germany"
# -------------------------------------------------------------
if "location" in dfc.columns:
    # Make sure we work with strings and replace NaN with empty string
    loc = dfc["location"].fillna("").astype(str)

    # Split into up to 3 parts by comma: p0, p1, p2
    parts = loc.str.split(",", n=2, expand=True).fillna("")

    # Ensure we always have exactly 3 columns (p0, p1, p2)
    while parts.shape[1] < 3:
        parts[parts.shape[1]] = ""

    # Strip whitespace in each part
    parts = parts.apply(lambda s: s.str.strip())
    parts.columns = ["p0", "p1", "p2"]

    # Count how many non-empty parts we have for each row
    cnt = (parts != "").sum(axis=1)

    # Initialize city / region / country
    city    = parts["p0"]
    region  = pd.Series([""] * len(parts), index=parts.index, dtype=object)
    country = pd.Series([""] * len(parts), index=parts.index, dtype=object)

    # Case 2: exactly two parts → city, country (no explicit region)
    mask2 = (cnt == 2)
    country.loc[mask2] = parts.loc[mask2, "p1"]

    # Case 3+: three parts → city, region, country
    mask3 = (cnt >= 3)
    region.loc[mask3]  = parts.loc[mask3, "p1"]
    country.loc[mask3] = parts.loc[mask3, "p2"]

    # Drop original 'location' and add new normalized columns
    dfc = dfc.drop(columns=["location"])
    dfc[["city", "region", "country"]] = pd.concat([city, region, country], axis=1)

# Quick sanity check
try:
    display(dfc[["city", "region", "country"]].head(10))
except:
    print(dfc[["city", "region", "country"]].head(10))

,city,region,country
0,Germany,,
1,Berlin,Berlin,Germany
2,Frankfurt Rhine-Main Metropolitan Area,,
3,North Rhine-Westphalia,,Germany
4,Stuttgart,Baden-Württemberg,Germany
5,Augsburg,Bavaria,Germany
6,Berlin,Berlin,Germany
7,Munich,Bavaria,Germany
8,Lähden,Lower Saxony,Germany
9,Berlin,Berlin,Germany


In [ ]:
if "postedTime" in dfc.columns:
    dfc = dfc.drop(columns=["postedTime"])

In [ ]:
# -------------------------------------------------------------
# General-purpose cleaners for remaining fields
# -------------------------------------------------------------

def clean_str_nan(x):
    """
    Normalize string-like fields:
    - convert 'nan', 'none', 'null', 'na', 'n/a' → empty string
    - strip whitespace
    Keeps numbers untouched.
    """
    if pd.isna(x):
        return ""
    s = str(x).strip()
    if s.lower() in ["nan", "none", "null", "na", "n/a"]:
        return ""
    return s


# -------------------------------------------------------------
# Apply basic cleaning to specific LinkedIn columns if present
# -------------------------------------------------------------
for col in ["posterProfileUrl", "posterFullName", "salary"]:
    if col in dfc.columns:
        dfc[col] = dfc[col].apply(clean_str_nan)


# -------------------------------------------------------------
# Clean experienceLevel (LinkedIn sometimes adds "level")
# Example: "Mid Level" → "Mid"
# -------------------------------------------------------------
if "experienceLevel" in dfc.columns:
    dfc["experienceLevel"] = (
        dfc["experienceLevel"]
        .apply(clean_str_nan)
        .str.replace("level", "", case=False, regex=False)
        .str.strip()
    )


# -------------------------------------------------------------
# Normalize applicationsCount
# LinkedIn formats could be:
#   "Be among the first 25 applicants"
#   "Over 200 applicants"
#   "30 applicants"
#   "Über 100 Bewerbungen"
# This function converts them into integers where possible.
# -------------------------------------------------------------
def clean_applications_count(x):
    if pd.isna(x) or str(x).strip() == "":
        return ""

    s = str(x).lower()

    # "Be among the first 25 applicants" → 25
    if "first 25" in s:
        return 25

    # "over 200 applicants" or "Über 200"
    match_over = re.search(r"(over|über)\s*(\d+)", s)
    if match_over:
        return int(match_over.group(2)) + 1

    # general numeric extraction
    match = re.search(r"\d+", s.replace(".", ""))
    if match:
        return int(match.group())

    return ""


if "applicationsCount" in dfc.columns:
    dfc["applicationsCount"] = dfc["applicationsCount"].apply(clean_applications_count)


# -------------------------------------------------------------
# Final clean-up: replace all 'nan-like' texts across dataframe
# -------------------------------------------------------------
dfc = dfc.applymap(
    lambda x: "" if str(x).strip().lower() in ["nan", "none", "null", "na", "n/a"] else x
)

/tmp/ipython-input-3329588626.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfc = dfc.applymap(


In [ ]:
# --- Extract analytics-related programs/tools from job descriptions and explode ---

import re

# Patterns for concrete analytics tools (BI, databases, languages, ETL, cloud, etc.)
PROGRAM_PATTERNS = [
    # BI / Reporting
    (r"\bpower\s*bi\b", "Power BI"),
    (r"\btableau\b", "Tableau"),
    (r"\blooker\b|\blooker\s*studio\b", "Looker"),
    (r"\bqlik(?:view|sense)?\b", "Qlik"),
    (r"\bpower\s*query\b", "Power Query"),
    (r"\bdax\b", "DAX"),
    (r"\bssrs\b", "SSRS"),

    # Spreadsheets
    (r"\bexcel\b", "Excel"),
    (r"\bgoogle\s*sheets?\b", "Google Sheets"),

    # Databases / SQL engines
    (r"\bms\s*sql\s*server\b|\bsql\s*server\b", "SQL Server"),
    (r"\bpostgres(?:ql)?\b", "PostgreSQL"),
    (r"\bmysql\b", "MySQL"),
    (r"\boracle\b", "Oracle"),
    (r"\bsnowflake\b", "Snowflake"),
    (r"\bbigquery\b", "BigQuery"),
    (r"\bredshift\b", "Redshift"),
    (r"\bsqlite\b", "SQLite"),
    (r"\bteradata\b", "Teradata"),
    (r"\bclickhouse\b", "ClickHouse"),

    # Data platforms / warehouses / lakehouses
    (r"\bdatabricks\b", "Databricks"),
    (r"\bdata\s*lake\b", "Data Lake"),
    (r"\bdata\s*warehouse\b", "Data Warehouse"),

    # Data integration / ETL / Orchestration
    (r"\bairflow\b", "Apache Airflow"),
    (r"\bfivetran\b", "Fivetran"),
    (r"\bmatillion\b", "Matillion"),
    (r"\binformatica\b", "Informatica"),
    (r"\bssis\b", "SSIS"),
    (r"\btalend\b", "Talend"),
    (r"\bdbt\b", "dbt"),

    # Programming languages for analytics
    (r"\bpython\b", "Python"),
    (r"\br\b", "R"),
    (r"\bscala\b", "Scala"),
    (r"\bmatlab\b", "MATLAB"),
    (r"\bsas\b", "SAS"),
    (r"\bstata\b", "Stata"),

    # Cloud platforms (as tools in the stack)
    (r"\bazure\b", "Azure"),
    (r"\baws\b", "AWS"),
    (r"\bamazon\s*web\s*services\b", "AWS"),
    (r"\bgcp\b|\bgoogle\s*cloud\b", "GCP"),
    (r"\bazure\s*synapse\b", "Azure Synapse"),
    (r"\bazure\s*data\s*factory\b", "Azure Data Factory"),

    # SAP analytics
    (r"\bsap\s*bw\b", "SAP BW"),
    (r"\bsap\s*hana\b", "SAP HANA"),

    # Other analytics-related tools
    (r"\bpower\s*pivot\b", "Power Pivot"),
    (r"\bknime\b", "KNIME"),
    (r"\brapidminer\b", "RapidMiner"),
]


def extract_programs_from_text(text: str):
    """
    Extract a list of analytics-related tools/programs from free text.
    Returns a list of unique labels in the order they appear.
    """
    if not isinstance(text, str) or not text.strip():
        return []
    t = text.lower()
    found = []

    for pat, label in PROGRAM_PATTERNS:
        if re.search(pat, t, flags=re.IGNORECASE):
            found.append(label)

    # Deduplicate while preserving order
    seen, uniq = set(), []
    for x in found:
        if x not in seen:
            seen.add(x)
            uniq.append(x)
    return uniq


# 1) Choose source column for text (prefer cleaned description)
if "description_clean" in dfc.columns:
    desc_col = "description_clean"
elif "description" in dfc.columns:
    desc_col = "description"
elif "descriptions" in dfc.columns:
    desc_col = "descriptions"
else:
    raise KeyError("No suitable description column found in dfc "
                   "(expected 'description_clean', 'description' or 'descriptions').")

# 2) Create a list of programs/tools per row
dfc["Programs"] = dfc[desc_col].apply(extract_programs_from_text)

# 3) Explode: one row per (job, program)
dfc = dfc.explode("Programs", ignore_index=True)

# 4) Drop empty / NaN programs
mask = dfc["Programs"].notna() & (dfc["Programs"].astype(str).str.strip() != "")
dfc = dfc[mask].reset_index(drop=True)

In [ ]:

!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
# EXTRACTION OF SKILLS/REQUIREMENTS AND PRETTY EXPORT FOR ANALYTICS ROLES

import re
import json
import pandas as pd

# Choose description column (prefer cleaned one if available)
if "description_clean" in dfc.columns:
    DESC_COL = "description_clean"
elif "description" in dfc.columns:
    DESC_COL = "description"
else:
    raise KeyError("Expected a description column: 'description_clean' or 'description'.")

# -------- dictionaries / patterns tuned for DATA / BI / ANALYTICS ROLES --------

HARD_SKILLS = {
    "sql / databases": [
        r"\bsql\b",
        r"\b(t[-\s]?sql|tsql)\b",
        r"\bpl/sql\b",
        r"\b(postgres(?:ql)?|postgres)\b",
        r"\bmysql\b",
        r"\boracle\b",
        r"\bsnowflake\b",
        r"\bbigquery\b",
        r"\bredshift\b",
        r"\bsql\s*server\b",
        r"\bsqlite\b",
        r"\bnosql\b",
        r"\bmongodb\b",
        r"\bdata\s*warehouse\b",
        r"\bdata\s*lake\b",
    ],
    "programming / scripting": [
        r"\bpython\b",
        r"\bpandas\b",
        r"\bnumpy\b",
        r"\bscikit[-\s]*learn\b",
        r"\b(py)?spark\b",
        r"\bpyspark\b",
        r"\br\b",
        r"\bscala\b",
        r"\bmatlab\b",
        r"\bsas\b",
        r"\bstata\b",
        r"\b(java(script)?|c\+\+?|go|rust)\b",
    ],
    "bi / reporting tools": [
        r"\bpower\s*bi\b",
        r"\btableau\b",
        r"\blooker\b",
        r"\blooker\s*studio\b",
        r"\bqlik(view|sense)?\b",
        r"\bssrs\b",
        r"\bssas\b",
        r"\b(cognos|microstrategy)\b",
        r"\bdashboards?\b",
        r"\b(kpi|okrs?)\b",
    ],
    "excel / spreadsheets": [
        r"\bexcel\b",
        r"\b(vlookup|index\s*\+?\s*match|pivot\s*table)s?\b",
        r"\b(spreadsheets?|google\s*sheets?)\b",
    ],
    "data modeling / warehousing": [
        r"\bdata\s*model(ling|ing)\b",
        r"\bdimensional\s*model(ling|ing)\b",
        r"\bstar\s*schema\b",
        r"\bsnowflake\s*schema\b",
        r"\b(etl|elt)\b",
        r"\bdata\s*pipeline(s)?\b",
        r"\bdbt\b",
    ],
    "etl / orchestration": [
        r"\betl\b",
        r"\belt\b",
        r"\bdata\s*integration\b",
        r"\bdata\s*ingestion\b",
        r"\bapache\s*airflow\b",
        r"\bairflow\b",
        r"\b(informatica|talend|matillion|fivetran)\b",
        r"\bss(is|is)\b",
    ],
    "statistics / maths": [
        r"\bstatistics\b",
        r"\bstatistical\b",
        r"\bhypothesis\s*testing\b",
        r"\b(a/b|ab)\s*testing\b",
        r"\bregression\b",
        r"\banova\b",
        r"\btime[-\s]*series\b",
        r"\bprobability\b",
        r"\blinear\s*algebra\b",
        r"\bmath(ematics)?\b",
    ],
    "machine learning / data science": [
        r"\bmachine\s*learning\b",
        r"\bml\b",
        r"\bdeep\s*learning\b",
        r"\b(neural\s*networks?|nn)\b",
        r"\bclustering\b",
        r"\bclassification\b",
        r"\bforecasting\b",
        r"\brecommendation\s*systems?\b",
    ],
    "cloud / big data": [
        r"\baws\b",
        r"\bazure\b",
        r"\bgcp\b",
        r"\bgoogle\s*cloud\b",
        r"\bamazon\s*web\s*services\b",
        r"\bazure\s*synapse\b",
        r"\bazure\s*data\s*factory\b",
        r"\bdatabricks\b",
        r"\bhadoop\b",
        r"\bspark\b",
        r"\bkafka\b",
    ],
    "version control / ci": [
        r"\bgit\b",
        r"\bgithub\b",
        r"\bgitlab\b",
        r"\bbitbucket\b",
        r"\bci/?cd\b",
        r"\bjenkins\b",
        r"\bdocker\b",
    ],
    "business / product analytics": [
        r"\bcohort\s*analysis\b",
        r"\bunit\s*economics\b",
        r"\bfunnel(s)?\b",
        r"\bretention\b",
        r"\bchurn\b",
        r"\bclick[-\s]*through\s*rate\b",
        r"\bconversion\s*rate\b",
        r"\bproduct\s*analytics?\b",
        r"\bpricing\s*analytics?\b",
    ],
}

SOFT_SKILLS = {
    "communication / stakeholder": [
        r"\bstakeholder(s)?\b",
        r"\bstakeholder\s*management\b",
        r"\bstakeholder\s*communication\b",
        r"\bcommunication\b",
        r"\bkommunikation\b",
        r"\bability to explain\b",
        r"\btranslate\b.*\bcomplex\b.*\binto\b.*\bsimple\b",
        r"\bstorytelling\b",
        r"\bdata\s*storytelling\b",
        r"\bpräsentationsfähigkeit\b",
    ],
    "problem solving / critical thinking": [
        r"\bproblem[-\s]*solving\b",
        r"\bcritical\s*thinking\b",
        r"\bstructured\s*(thinking|approach)\b",
        r"\banalytical\s*(mindset|skills?)\b",
        r"\bstrong analytical\b",
        r"\broot cause\b",
    ],
    "ownership / proactivity": [
        r"\bownership\b",
        r"\bproactive\b",
        r"\bself[-\s]*starter\b",
        r"\binitiative\b",
        r"\bhands[-\s]*on\b",
        r"\bcan[-\s]*do\b",
        r"\bverantwortungs(bewusstsein)?\b",
        r"\beigenständig\b",
        r"\bselbstständig\b",
    ],
    "business acumen / product thinking": [
        r"\bbusiness\s*acumen\b",
        r"\bcommercial\s*mindset\b",
        r"\bunderstanding of business\b",
        r"\bproduct\s*thinking\b",
        r"\bbusiness\s*partner\b",
        r"\bbetreibliches\s*verständnis\b",
    ],
    "self-organization / prioritization": [
        r"\bprioriti[sz]ation\b",
        r"\btime\s*management\b",
        r"\bself[-\s]*organized\b",
        r"\bself[-\s]*organisation\b",
        r"\borganized\b",
        r"\bstrukturierte\s*arbeitsweise\b",
    ],
    "teamwork / collaboration": [
        r"\bteamwork\b",
        r"\bteam\s*player\b",
        r"\bcollaborative\b",
        r"\bcross[-\s]*functional\b",
    ],
    "attention to detail / quality": [
        r"\battention to detail\b",
        r"\bdetail[-\s]*oriented\b",
        r"\bsorgfalt\b",
        r"\bgenauigkeit\b",
        r"\bquality[-\s]*driven\b",
    ],
    "adaptability / learning": [
        r"\badaptab(le|ility)\b",
        r"\bfast\s*learner\b",
        r"\bcurious(ity)?\b",
        r"\bcontinuous\s*learning\b",
        r"\bopen\s*to\s*feedback\b",
    ],
}

LANG_PATTERNS = {
    "en": [
        r"\benglish\b",
        r"\benglisch\b",
        r"\bfluent in english\b",
        r"\bc1 (english|englisch)\b",
    ],
    "de": [
        r"\bgerman\b",
        r"\bdeutsch\b",
        r"\bfließend(e[rsn]?)?\s*deutsch\b",
        r"\bc1 (german|deutsch)\b",
        r"\bverhandlungssicher(es)? deutsch\b",
    ],
}

DEGREE_MAP = {
    "bachelor": [
        r"\bbachelor('?s)?\b",
        r"\bbsc\b",
        r"\bb\.sc\.\b",
        r"\bdegree\b",
    ],
    "master": [
        r"\bmaster('?s)?\b",
        r"\bmsc\b",
        r"\bm\.sc\.\b",
    ],
    "data / quant degree": [
        r"\b(data\s*science|statistics?|mathematics?|computer\s*science)\b",
        r"\b(stem)\b",
        r"\bquant(itative)?\b",
    ],
    "ausbildung": [
        r"\babgeschlossen(e|es)?(r)? studium\b",
        r"\bausbildung\b",
        r"\bfachinformatiker\b",
        r"\bwirtschaftsinformatik\b",
        r"\babitur\b",
    ],
}

EXP_PATTERNS = [
    r"(\d{1,2})\s*\+?\s*(years?|yrs?)\b",
    r"mind\.?\s*(\d{1,2})\s*jahre\b",
    r"über\s*(\d{1,2})\s*j(ahre)?\b",
    r"(\d{1,2})\s*j(ahre)?\s*erfahrung\b",
]

WORKLOAD_PATTERNS = [
    r"\b(20|25|30|32|35|38|40)\s*(std\.?|stunden)\b",
    r"\b(20-?30|20\s*-\s*30)\s*stunden\b",
    r"\b70\s*[-–]\s*80 ?%\b",
    r"\bteilzeit\b",
    r"\bvollzeit\b",
    r"\bpart[- ]time\b",
    r"\bfull[- ]time\b",
    r"\bhybrid\b",
    r"\bremote\b",
    r"\bon[- ]site\b",
    r"\b3-4\s*tage\b",
]

LOCATION_PATTERNS = [
    r"\bberlin\b",
    r"\bmünchen\b",
    r"\bmuenchen\b",
    r"\bfrankfurt\b",
    r"\bhamburg\b",
    r"\bköln\b",
    r"\bkoeln\b",
    r"\bd(ü|u)sseldorf\b",
    r"\bessen\b",
    r"\bstuttgart\b",
    r"\bnürnberg\b",
    r"\bmunich\b",
    r"\bgermany\b",
    r"\bdeutschland\b",
    r"\blocation\s*:\s*([^\n\r]+)",
]

INDUSTRY_PATTERNS = {
    "finance/consulting": [
        r"\bfinancial services?\b",
        r"\bprivate equity\b",
        r"\bconsult(ing|ancy)\b",
        r"\binvestment bank\b",
        r"\basset management\b",
    ],
    "saas/tech": [
        r"\bsaas\b",
        r"\b(start[- ]?up|scale[- ]?up)\b",
        r"\bsoftware\s*company\b",
        r"\btech company\b",
    ],
    "retail/e-commerce": [
        r"\b(retail|e-?commerce)\b",
        r"\bonline shop\b",
        r"\bomnichannel\b",
    ],
    "energy/green": [
        r"\bsolar\b",
        r"\b(battery|batteries)\b",
        r"\b(e-)?mobility\b",
        r"\brenewable\b",
    ],
    "healthcare/pharma": [
        r"\bpharma(ceutical)?\b",
        r"\bmedical devices?\b",
        r"\bhealthcare\b",
    ],
}

# -------- utility functions (same structure, just reused) --------

def _norm(s: str) -> str:
    s = re.sub(r"[ \t]+", " ", str(s))
    s = re.sub(r"\s+\n", "\n", s)
    s = re.sub(r"\n\s+", "\n", s)
    return s.strip()

def _found(pats, text):
    return any(re.search(p, text, flags=re.IGNORECASE) for p in pats)

def _collect(dic, text):
    seen, out = set(), []
    for k, pats in dic.items():
        if _found(pats, text) and k not in seen:
            seen.add(k)
            out.append(k)
    return out

def _langs(text):
    res = []
    for code, pats in LANG_PATTERNS.items():
        if _found(pats, text):
            res.append(code)
    # deduplicate preserving order
    return list(dict.fromkeys(res))

def _degrees(text):
    hits = []
    for name, pats in DEGREE_MAP.items():
        if _found(pats, text):
            hits.append(name)
    return list(dict.fromkeys(hits))

def _years(text):
    for p in EXP_PATTERNS:
        m = re.search(p, text, flags=re.IGNORECASE)
        if m:
            try:
                return int(m.group(1))
            except Exception:
                pass
    return None

def _list_by_pats(pats, text):
    vals, seen = [], set()
    for p in pats:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            hit = m.group(0)
            if hit not in seen:
                seen.add(hit)
                vals.append(hit)
    return vals

def _locations_from_text(text):
    vals = []
    for p in LOCATION_PATTERNS:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            vals.append(m.group(1) if m.lastindex else m.group(0))
    vals = [re.sub(r"^(location\s*:\s*)", "", v, flags=re.IGNORECASE).strip(" :") for v in vals]
    vals = [v.title() for v in vals]
    seen, out = set(), []
    for v in vals:
        if v and v not in seen:
            seen.add(v)
            out.append(v)
    return out

def _industry(text):
    for name, pats in INDUSTRY_PATTERNS.items():
        if _found(pats, text):
            return name
    return ""

# --- Description sections classification (still useful for analytics JDs) ---
SECTION_HINTS = {
    "responsibilities": [
        r"\b(your|deine|ihre)\s+(tasks|aufgaben|mission)\b",
        r"\bkey responsibilities\b",
        r"\bwhat you will do\b",
    ],
    "requirements": [
        r"\bwhat you bring\b",
        r"\brequirements?\b",
        r"\byour profile\b",
        r"\b(dein|ihr)e(s)? profil\b",
        r"\bqualifikationen?\b",
        r"\bqualifications?\b",
        r"\bskills?\b",
    ],
    "benefits": [
        r"\bbenefits?\b",
        r"\bwhat'?s on offer\b",
        r"\bwhy (us|verimi|enpal|tipico)\b",
        r"\bwir bieten\b",
        r"\bdas erwartet dich\b",
    ],
    "about_company": [
        r"\babout (us|the firm|company)\b",
        r"\bwer wir sind\b",
        r"\bunternehmensbeschreibung\b",
        r"\büber (uns|verimi|enpal|mcmakler)\b",
        r"\bwho are we\?\b",
        r"\bshort facts\b",
    ],
}

def _classify_lines(text):
    t = text.replace("\r", "\n")
    # split on newlines, bullets, numbered lists
    chunks = re.split(r"\n|(?:^|\n)\s*[•*–-]\s*|(?:^|\n)\s*\d+\.\s*", t)
    lines = [_norm(x) for x in chunks if _norm(x)]
    current = "other"
    buckets = {"about_company": [], "responsibilities": [], "requirements": [], "benefits": [], "other": []}
    for ln in lines:
        lowered = ln.lower()
        switched = False
        for sec, pats in SECTION_HINTS.items():
            if any(re.search(p, lowered) for p in pats):
                current = sec
                switched = True
                break
        if switched:
            continue
        buckets[current].append(ln)
    for k in buckets:
        buckets[k] = [x for x in buckets[k] if x]
    return buckets

def parse_description(text: str) -> dict:
    t = _norm(text)
    sections = _classify_lines(t)
    return {
        "hard_skills": _collect(HARD_SKILLS, t),
        "soft_skills": _collect(SOFT_SKILLS, t),
        "years_experience": _years(t),
        "degrees": _degrees(t),
        "languages": _langs(t),
        "workload_mentions": _list_by_pats(WORKLOAD_PATTERNS, t),
        "locations_in_text": _locations_from_text(t),
        "industry": _industry(t),
        **{f"{k}_lines": v for k, v in sections.items()},
        "preview": t[:300],
    }

# ---- Apply to cleaned dfc ----
parsed = dfc[DESC_COL].fillna("").apply(parse_description)

# ---- Flatten parsed dicts into columns ----
skills_df = parsed.apply(
    lambda x: pd.Series(
        {
            "hard_skills": json.dumps(x["hard_skills"], ensure_ascii=False),
            "soft_skills": json.dumps(x["soft_skills"], ensure_ascii=False),
            "years_experience": x["years_experience"],
            "degrees": json.dumps(x["degrees"], ensure_ascii=False),
            "languages": json.dumps(x["languages"], ensure_ascii=False),
            "workload_mentions": json.dumps(x["workload_mentions"], ensure_ascii=False),
            "locations_in_text": json.dumps(x["locations_in_text"], ensure_ascii=False),
            "industry": x["industry"],
            "about_company_lines": json.dumps(x.get("about_company_lines", []), ensure_ascii=False),
            "responsibilities_lines": json.dumps(x.get("responsibilities_lines", []), ensure_ascii=False),
            "requirements_lines": json.dumps(x.get("requirements_lines", []), ensure_ascii=False),
            "benefits_lines": json.dumps(x.get("benefits_lines", []), ensure_ascii=False),
            "other_lines": json.dumps(x.get("other_lines", []), ensure_ascii=False),
            "preview": x["preview"],
        }
    )
)

# ---- Merge with dfc ----
enriched = pd.concat([dfc.reset_index(drop=True), skills_df.reset_index(drop=True)], axis=1)

# ---- Pretty Excel export (no JSON brackets in cells) ----
def _to_list(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        s = val.strip()
        if s == "" or s.lower() in {"nan", "none", "null", "na", "n/a"}:
            return []
        try:
            j = json.loads(s)
            if isinstance(j, list):
                return j
        except Exception:
            # fallback: split by common separators
            return [p.strip() for p in re.split(r"[;,\n]\s*", s) if p.strip()]
    return []

def pretty_list(val):
    return "; ".join(_to_list(val))

def pretty_bullets(val):
    items = _to_list(val)
    return "" if not items else "• " + "\n• ".join(items)

pretty = enriched.copy()

for c in ["hard_skills", "soft_skills", "languages", "degrees", "workload_mentions", "locations_in_text"]:
    if c in pretty.columns:
        pretty[c] = pretty[c].apply(pretty_list)

for c in [
    "about_company_lines",
    "responsibilities_lines",
    "requirements_lines",
    "benefits_lines",
    "other_lines",
    "preview",
    DESC_COL,
]:
    if c in pretty.columns:
        # keep preview/description as plain text, others → bullets
        if c in ["preview", DESC_COL]:
            continue
        pretty[c] = pretty[c].apply(pretty_bullets)

out_pretty = "analytics_skills_pretty.xlsx"

with pd.ExcelWriter(out_pretty, engine="xlsxwriter") as writer:
    pretty.to_excel(writer, index=False, sheet_name="data")
    wb, ws = writer.book, writer.sheets["data"]
    wrap = wb.add_format({"text_wrap": True, "valign": "top"})
    # Base width
    ws.set_column(0, pretty.shape[1] - 1, 18)
    # Long text columns — wider with wrap
    for col_name in [
        c
        for c in ["description_clean", "description", "preview",
                  "about_company_lines", "responsibilities_lines",
                  "requirements_lines", "benefits_lines", "other_lines"]
        if c in pretty.columns
    ]:
        idx = pretty.columns.get_loc(col_name)
        ws.set_column(idx, idx, 45, wrap)
    # Skill lists — medium width with wrap
    for col_name in [
        c
        for c in ["hard_skills", "soft_skills", "languages", "degrees",
                  "workload_mentions", "locations_in_text"]
        if c in pretty.columns
    ]:
        idx = pretty.columns.get_loc(col_name)
        ws.set_column(idx, idx, 28, wrap)

In [ ]:
# -------------------------------------------------------------
# Final salary cleaning: convert string → float (EUR, yearly)
# -------------------------------------------------------------

for col in ["salary_min (€)", "salary_max (€)"]:
    if col in dfc.columns:
        dfc[col] = (
            dfc[col]
            .astype(str)
            .str.replace("€", "", regex=False)          # remove euro symbol
            .str.replace(r"[^\d.,-]", "", regex=True)   # keep only digits, dots, commas and dashes
            .str.replace(",", ".", regex=False)         # convert comma → dot
            .replace("", np.nan)                        # empty strings → NaN
            .astype(float)                              # safe conversion to float
        )

# Rename columns to clean SQL-friendly names
dfc = dfc.rename(columns={
    "salary_min (€)": "salary_min",
    "salary_max (€)": "salary_max"
})

In [ ]:
import re
import pandas as pd

# 1) Find description column
desc_col = next(
    (c for c in ["description_clean", "description", "descriptions", "job_description", "desc"]
     if c in dfc.columns),
    None
)
if desc_col is None:
    raise ValueError("No description column found. Expected 'description_clean' or 'description'.")

# 2) Patterns ONLY for Data Analyst / BI / Business Analyst tools
PROGRAM_PATTERNS = [
    # BI tools
    (r"\bpower\s*bi\b", "Power BI"),
    (r"\btableau\b", "Tableau"),
    (r"\blooker\b|\blooker\s*studio\b", "Looker"),
    (r"\bqlik(view|sense)?\b", "Qlik"),

    # SQL + Databases
    (r"\bsql\b", "SQL"),
    (r"\bmysql\b", "MySQL"),
    (r"\bpostgres(?:ql)?\b", "PostgreSQL"),
    (r"\bsql\s*server\b|\bms\s*sql\b", "SQL Server"),
    (r"\boracle\b", "Oracle DB"),
    (r"\bsnowflake\b", "Snowflake"),
    (r"\bbigquery\b", "BigQuery"),
    (r"\bredshift\b", "Redshift"),
    (r"\bsqlite\b", "SQLite"),

    # Python / R
    (r"\bpython\b", "Python"),
    (r"\bpandas\b", "Pandas"),
    (r"\bnumpy\b", "NumPy"),
    (r"\bscikit[-\s]*learn\b", "Scikit-Learn"),
    (r"\bpyspark\b", "PySpark"),
    (r"\br\b", "R"),

    # Cloud
    (r"\baws\b|\bamazon\s*web\s*services\b", "AWS"),
    (r"\bazure\b", "Azure"),
    (r"\bgcp\b|\bgoogle\s*cloud\b", "GCP"),
    (r"\bazure\s*data\s*factory\b", "Azure Data Factory"),
    (r"\bazure\s*synapse\b", "Azure Synapse"),
    (r"\bdatabricks\b", "Databricks"),

    # ETL / Orchestration
    (r"\bairflow\b", "Airflow"),
    (r"\betl\b", "ETL"),
    (r"\belt\b", "ELT"),
    (r"\bfivetran\b", "Fivetran"),
    (r"\bmatillion\b", "Matillion"),
    (r"\binformatica\b", "Informatica"),
    (r"\btalend\b", "Talend"),
    (r"\bdbt\b", "dbt"),

    # Version control
    (r"\bgit\b", "Git"),
    (r"\bgithub\b", "GitHub"),
    (r"\bgitlab\b", "GitLab"),

    # Spreadsheets
    (r"\bexcel\b", "Excel"),
    (r"\bgoogle\s*sheets\b", "Google Sheets")
]


def extract_programs_from_text(text: str):
    """Extract analytics tools from the description text."""
    if not isinstance(text, str) or not text.strip():
        return []
    t = text.lower()
    found = []

    for pat, label in PROGRAM_PATTERNS:
        if re.search(pat, t, flags=re.IGNORECASE):
            found.append(label)

    # unique, preserve order
    seen, uniq = set(), []
    for x in found:
        if x not in seen:
            seen.add(x)
            uniq.append(x)
    return uniq

# 3) Create Programs column
dfc["Programs"] = dfc[desc_col].fillna("").apply(extract_programs_from_text)

# 4) Explode to one tool per row
dfc = dfc.explode("Programs", ignore_index=True)

# 5) Drop empty values
dfc = dfc[dfc["Programs"].notna() & (dfc["Programs"].str.strip() != "")]

print(f"Строк после взрыва Programs: {len(dfc)}")
print(dfc["Programs"].value_counts().head(10))

Строк после взрыва Programs: 6526
Programs
SQL          1016
Python        818
Tableau       764
Power BI      708
R             459
Excel         372
Looker        245
Snowflake     232
Qlik          210
ETL           206
Name: count, dtype: int64


In [ ]:
import pandas as pd
import json

# Source file exported from previous step (analytics_skills_pretty.xlsx)
src_file = "analytics_skills_pretty.xlsx"
df = pd.read_excel(src_file)

# List of columns that contain lists (either as ';' separated strings or JSON)
list_cols = [
    "hard_skills", "soft_skills", "languages", "degrees",
    "workload_mentions", "locations_in_text",
    "about_company_lines", "responsibilities_lines",
    "requirements_lines", "benefits_lines", "other_lines"
]

rows = []

for idx, row in df.iterrows():
    # Convert each "list-like" column into a proper Python list
    expanded_lists = {}
    for col in list_cols:
        val = row.get(col, "")
        if pd.isna(val) or str(val).strip() == "":
            expanded_lists[col] = []
        else:
            # If it's JSON (starts with '[') → parse JSON
            # Otherwise → split by ';'
            try:
                if isinstance(val, str) and val.strip().startswith("["):
                    expanded_lists[col] = json.loads(val)
                else:
                    expanded_lists[col] = [
                        v.strip() for v in str(val).split(";") if v.strip()
                    ]
            except Exception:
                expanded_lists[col] = [
                    v.strip() for v in str(val).split(";") if v.strip()
                ]

    # Determine max length of all lists for this row
    max_len = max(len(expanded_lists[c]) for c in list_cols)

    # Create max_len rows, duplicating scalar fields
    for i in range(max_len if max_len > 0 else 1):
        new_row = row.to_dict()
        for col in list_cols:
            items = expanded_lists[col]
            new_row[col] = items[i] if i < len(items) else None
        rows.append(new_row)

df_exploded = pd.DataFrame(rows)

out_file = "analytics_jobs_exploded_2025-11-19.xlsx"
df_exploded.to_excel(out_file, index=False)

print(f"Exploded file: {out_file}, rows: {len(df_exploded)}")

Exploded file: analytics_jobs_exploded_2025-11-19.xlsx, rows: 5896
